In [ ]:
import numpy as np
import codecs
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Input
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.models import Model
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [ ]:
path = '/content/drive/My Drive/Sentiment Analysis/'
def load_embeddings(embedding_path, glove_len):
  weight_vectors = [np.zeros((glove_len, ))]
  word_idx = {}
  with codecs.open(embedding_path, encoding='utf-8') as f:
    for line in f:
      word, vec = line.split(u' ', 1)
      word_idx[word.lower()] = len(weight_vectors)
      weight_vectors.append(np.array(vec.split(), dtype=np.float32))
  word_idx[u'-LRB-'] = word_idx.pop(u'(')
  word_idx[u'-RRB-'] = word_idx.pop(u')')
  weight_vectors.append(np.random.uniform(
      -0.05, 0.05, weight_vectors[0].shape).astype(np.float32))
  return np.asarray(np.stack(weight_vectors)), word_idx

emb_matrix, word_idx = load_embeddings(path+'Data/glove_6B_100d.txt', 100)
max_len = 56

In [ ]:
X_train = np.load(path+'Data/X_train.npy')
X_test = np.load(path+'Data/X_test.npy')
X_dev = np.load(path+'Data/X_dev.npy')
Y_train = np.load(path+'Data/Y_train.npy')
Y_test = np.load(path+'Data/Y_test.npy')
Y_dev = np.load(path+'Data/Y_dev.npy')
X_train = np.concatenate((X_train, X_test), axis=0)
Y_train = np.concatenate((Y_train, Y_test), axis=0)
print(Y_train.shape)

(179247, 10)


In [ ]:
def pretrained_embedding_layer(emb_matrix):
    embedding_layer = Embedding(emb_matrix.shape[0],emb_matrix.shape[1], trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

def make_model(input_shape, emb_matrix):
    phrase_indices = Input(shape=input_shape, dtype = 'int32')
    emb_layer = pretrained_embedding_layer(emb_matrix)
    embeddings = emb_layer(phrase_indices)   
    X = Bidirectional(LSTM(128, return_sequences=True))(embeddings)
    X = Bidirectional(LSTM(128))(X)
    X = Dense(512, activation='relu')(X)
    X = Dense(10, activation='softmax')(X)
    model = Model(inputs=phrase_indices, outputs=X)
    return model

model = make_model((max_len,), emb_matrix)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
checkpoint_path = "/content/drive/My Drive/Sentiment Analysis/Data/cpnew11.ckpt"
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=False)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 56)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 56, 100)           40000200  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 56, 256)           234496    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 40,765,650
Trainable params: 765,450
Non-trainable params: 40,000,200
_________________________________________

In [ ]:
model.fit(X_train, Y_train, epochs = 20, batch_size = 3000, shuffle=True, callbacks=[cp_callback])

Epoch 1/20
179247/179247 [==============================] - 65s 360us/step - loss: 1.7694 - accuracy: 0.3515

Epoch 00001: saving model to /content/drive/My Drive/Sentiment Analysis/Data/cpnew11.ckpt
Epoch 2/20
179247/179247 [==============================] - 62s 345us/step - loss: 1.5700 - accuracy: 0.3958

Epoch 00002: saving model to /content/drive/My Drive/Sentiment Analysis/Data/cpnew11.ckpt
Epoch 3/20
179247/179247 [==============================] - 62s 345us/step - loss: 1.5280 - accuracy: 0.4056

Epoch 00003: saving model to /content/drive/My Drive/Sentiment Analysis/Data/cpnew11.ckpt
Epoch 4/20
179247/179247 [==============================] - 62s 344us/step - loss: 1.5057 - accuracy: 0.4124

Epoch 00004: saving model to /content/drive/My Drive/Sentiment Analysis/Data/cpnew11.ckpt
Epoch 5/20
179247/179247 [==============================] - 62s 345us/step - loss: 1.4910 - accuracy: 0.4147

Epoch 00005: saving model to /content/drive/My Drive/Sentiment Analysis/Data/cpnew11.ckpt


KeyboardInterrupt: ignored

In [ ]:
model.save_weights("/content/drive/My Drive/Sentiment Analysis/Data/model11.h5")

In [ ]:
weight_path = '/content/drive/My Drive/Sentiment Analysis/Data/model11.h5'
loaded_model = make_model((max_len,), emb_matrix)
loaded_model.load_weights(weight_path)
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
loss_dev, acc_dev = loaded_model.evaluate(X_dev, Y_dev)
print(loss_dev, acc_dev)

59985/59985 [==============================] - 64s 1ms/step
1.5040373155988633 0.4160206615924835
